# Case Studies
Taking a closer look at specific tracts/neighborhoods as they relate to features and model predictions

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import geopandas as gpd

import sys
sys.path.insert(0, './')
from functions import *
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [33]:
df = pd.read_parquet("Data/Cleaned/after_kriging.parquet")
features = pd.read_parquet("Data/Cleaned/post_feat_engineering.parquet")
labels = pd.read_parquet("Data/Cleaned/labels_may5.parquet")
future_preds = pd.read_parquet("Data/Cleaned/future_preds.parquet")


## Baselines
First we look at averages across NYC so we have something to compare to

In [34]:
df.groupby('year')[['percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
        'num_evictions_commercial', 'num_evictions_residential']].mean()

,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
year,,,,,,,,,,,,,,,,,,,,
2010,0.242454,0.139965,0.043005,0.016762,0.449438,0.261382,0.124545,0.259714,0.131002,0.088334,0.171134,55528.931168,1156.765636,36.157028,535086.965243,5.938065,0.000000,266.120218,0.418319,5.535434
2011,0.242674,0.137526,0.043336,0.017122,0.448856,0.261790,0.124914,0.259941,0.132909,0.089879,0.170348,57111.915472,1208.907849,36.231290,540994.115973,6.155269,0.000000,269.280613,0.430089,5.969503
2012,0.243551,0.135903,0.043124,0.017312,0.449617,0.261391,0.126017,0.263949,0.134143,0.090371,0.168931,57716.935201,1247.218521,36.327653,534522.202972,6.540215,0.000000,272.592358,0.442505,6.402725
2013,0.242832,0.134236,0.042204,0.017723,0.447559,0.259503,0.127624,0.263208,0.136295,0.092490,0.167461,58449.423601,1279.950349,36.508370,526466.242248,7.267957,4822.243871,275.941954,0.454796,6.836497
2014,0.246214,0.133881,0.042324,0.018007,0.441837,0.258481,0.130972,0.264097,0.140190,0.095211,0.167109,59330.267990,1311.083148,36.623911,526811.641482,7.753118,6951.583656,279.114069,0.467236,7.270738
2015,0.250815,0.134240,0.043164,0.017944,0.437897,0.256035,0.134386,0.264510,0.143412,0.098101,0.165452,60179.990343,1353.480023,36.761491,561132.415163,8.412043,8548.784516,282.848014,0.479840,7.704377
2016,0.253764,0.135691,0.043299,0.018642,0.436232,0.252976,0.137392,0.266742,0.146280,0.100864,0.162704,62185.511682,1406.594339,36.915523,580909.165138,9.189677,11910.135914,285.629622,0.492971,8.137841
2017,0.255537,0.135560,0.043058,0.018995,0.435236,0.251176,0.140188,0.267264,0.148578,0.103868,0.159853,65067.340476,1456.235998,37.334528,609065.251468,10.030968,14076.883441,289.011500,0.506015,8.575314
2018,0.256467,0.136981,0.043575,0.019341,0.431086,0.250792,0.141076,0.269538,0.151588,0.106136,0.157048,68239.983310,1502.724477,37.654115,646815.258788,10.758710,15095.210323,292.426550,0.540951,8.239237


In [35]:
features.groupby('pred_year')[['percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
        'num_evictions_commercial', 'num_evictions_residential']].mean()

,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
pred_year,,,,,,,,,,,,,,,,,,,,
2020,0.018113,0.006759,0.092007,0.132556,0.054645,0.138879,0.170410,0.063371,0.044026,0.090245,0.024107,0.024688,0.037249,0.007308,0.045150,0.063098,0.318214,0.016976,0.032198,0.077127
2021,0.016786,0.011739,0.088805,0.139968,0.049052,0.125641,0.169037,0.058854,0.045983,0.085219,0.017563,0.027360,0.034968,0.008322,0.045900,0.064160,0.239587,0.015174,-0.019618,0.030259
2022,0.015305,0.018323,0.089434,0.146794,0.038535,0.126863,0.168381,0.049784,0.042426,0.081854,0.013209,0.033533,0.036507,0.007786,0.056679,0.073975,0.340944,0.007047,-0.042693,0.027974
2023,0.015955,0.020917,0.077293,0.136494,0.036942,0.125927,0.159893,0.060919,0.045196,0.076418,0.008756,0.040552,0.038460,0.008167,0.063017,0.084070,0.197187,0.005178,-0.133215,0.011284
2024,0.002244,0.021928,0.070475,0.315161,0.029528,0.119090,0.133687,0.056693,0.044081,0.075320,0.006131,0.045995,0.040661,0.008433,0.072927,0.089004,1.234085,0.003492,-0.264729,-0.127031
2025,-0.013264,0.022502,0.057796,0.414669,0.015978,0.116558,0.132577,0.052062,0.040223,0.072535,0.005295,0.055950,0.045698,0.008995,0.069112,0.087648,5.068263,0.001726,-0.329470,-0.238397
2026,-0.022915,0.021602,0.059805,0.456526,-0.002793,0.115251,0.169778,0.063214,0.039746,0.072725,0.006882,0.065355,0.053235,0.009548,0.088390,0.076699,4.122435,0.000586,-0.005809,0.149221
2027,-0.028600,0.021810,0.066574,0.474557,-0.017343,0.120716,0.143567,0.067232,0.037700,0.076038,0.014038,0.066232,0.054831,0.010085,0.105279,0.078682,42.643693,-0.001285,0.006161,0.406266


In [64]:
df_bk = df[df.GEOID=='36047122000']  # BK (East NY)
df_bk[['year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
1161,2010,0.245577,0.155192,0.003654,0.000000,0.086154,0.779808,0.000000,0.192500,0.089038,0.020577,0.112885,39945.0,991.0,27.0,551100.0,0.0,0.0,476.0,0.0,4.875
1161,2011,0.252670,0.152263,0.004467,0.000000,0.069334,0.810449,0.013983,0.187027,0.073607,0.025442,0.099631,41645.0,1097.0,28.1,568300.0,0.0,0.0,537.0,0.0,9.750
1161,2012,0.279303,0.150671,0.005209,0.007614,0.074935,0.847926,0.014226,0.165698,0.054498,0.035865,0.099980,43214.0,1133.0,29.1,572100.0,0.0,0.0,598.0,0.0,14.625
1161,2013,0.294379,0.126294,0.003513,0.005917,0.092271,0.786982,0.014423,0.205436,0.033099,0.026997,0.104290,40690.0,1125.0,28.4,585900.0,0.0,0.0,660.0,0.0,19.500
1161,2014,0.293139,0.118735,0.000000,0.006288,0.079896,0.803588,0.010357,0.213057,0.058998,0.012206,0.091178,38665.0,1139.0,30.3,587300.0,0.0,0.0,721.0,0.0,24.375
1161,2015,0.273641,0.160057,0.000000,0.005383,0.058496,0.791674,0.011843,0.221245,0.106585,0.025839,0.135654,43125.0,1184.0,33.4,564400.0,0.0,0.0,783.0,0.0,29.250
1161,2016,0.253261,0.186007,0.007623,0.010673,0.050313,0.834999,0.005929,0.184991,0.110452,0.034898,0.154498,44930.0,1240.0,31.3,572000.0,0.0,0.0,844.0,0.0,34.125
1161,2017,0.256606,0.191441,0.006484,0.011185,0.079429,0.823472,0.006160,0.175393,0.105041,0.027071,0.175717,44944.0,1241.0,31.2,554200.0,10.0,0.0,905.0,0.0,39.000
1161,2018,0.199636,0.183621,0.008553,0.011283,0.054413,0.894813,0.008189,0.133212,0.106096,0.040764,0.181256,39848.0,1330.0,33.1,575600.0,10.0,0.0,967.0,0.0,30.000
1161,2019,0.201514,0.174264,0.018167,0.009924,0.060555,0.890833,0.010765,0.107822,0.088814,0.041548,0.143314,39674.0,1283.0,31.8,580900.0,35.0,0.0,1028.0,0.0,39.000


In [65]:
df_bk = features[features.GEOID=='36047122000']  # BK (East NY)
df_bk[['pred_year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
1161,2020,0.007177,0.043196,-0.234210,0.169635,-0.070825,0.012444,-0.121317,0.001502,0.133294,0.219732,0.071697,0.021544,0.038661,0.026435,0.006480,0.000000,0.0,0.100291,0.0,0.408333
3486,2021,0.004564,0.051211,-0.327199,0.145311,0.058163,0.003594,-0.094614,-0.002406,0.154015,0.142949,0.114156,0.014503,0.020969,0.019112,-0.003908,0.000000,0.0,0.090978,0.0,0.265476
5811,2022,-0.050005,0.046146,-0.288974,0.122553,-0.007793,0.010326,-0.042592,-0.023482,0.198956,0.158977,0.118825,-0.010674,0.027452,0.023331,0.001413,0.000000,0.0,0.083464,0.0,0.143681
8136,2023,-0.057434,0.064617,0.073405,0.139621,-0.027537,0.021564,0.007523,-0.095219,0.237250,0.203388,0.076752,-0.001667,0.022739,0.020794,-0.001072,1.250000,0.0,0.076698,0.0,0.138126
10461,2024,-0.070639,0.085829,0.312894,0.133448,-0.062049,0.026000,0.035263,-0.151584,0.035067,0.212989,0.031087,0.039987,0.014040,0.009644,0.005100,0.833333,0.0,0.071346,0.0,-0.044567
12786,2025,-0.054340,0.018533,0.207162,0.066068,0.122695,0.026361,-0.155314,-0.144532,-0.163844,0.007278,-0.030358,0.022235,0.014896,-0.014221,0.006602,0.625000,0.0,0.066341,0.0,-0.244567
15111,2026,-0.050000,-0.015291,0.124696,-0.264366,0.005822,0.019275,-0.086508,-0.163659,-0.196195,-0.102645,-0.140593,0.046552,0.009603,-0.001556,0.010163,0.500000,0.0,0.062189,0.0,-0.326813
17436,2027,-0.025434,-0.016666,0.193070,-0.326842,-0.151849,0.018326,0.675792,-0.196044,-0.191483,0.225556,-0.180586,0.085097,0.003220,-0.002103,0.025606,0.421429,-0.5,0.058670,0.0,0.273187


## Brooklyn - Bushwick (Gentrified)

In [19]:
df_bk = df[df.GEOID=='36047044300']  # BK (Bushwick)
df_bk[['year',
       'percent_wfh',  'percent_latino',
       'percent_bachelors','percent_not_citizen',
       'med_rent_acs','med_value',
    'num_citibike_rides', 'num_trees']]  #.to_csv('Data/Cleaned/case_study_bushwick.csv')

,year,percent_wfh,percent_latino,percent_bachelors,percent_not_citizen,med_rent_acs,med_value,num_citibike_rides,num_trees
766,2010,0.007747,0.717560,0.055520,0.404939,1089.0,507700.0,0.0,265.0
766,2011,0.006753,0.732111,0.058852,0.434153,1178.0,526200.0,0.0,270.0
766,2012,0.009683,0.732857,0.066508,0.460317,1253.0,461500.0,0.0,276.0
766,2013,0.007994,0.734973,0.068563,0.433666,1333.0,464400.0,0.0,281.0
766,2014,0.003227,0.724105,0.086157,0.419329,1380.0,500000.0,0.0,287.0
766,2015,0.014132,0.681480,0.114541,0.379509,1387.0,476100.0,0.0,293.0
766,2016,0.016069,0.668822,0.126154,0.340968,1408.0,525600.0,0.0,298.0
766,2017,0.022672,0.550449,0.155842,0.247345,1408.0,492600.0,0.0,304.0
766,2018,0.028121,0.548464,0.175708,0.266853,1484.0,628700.0,0.0,309.0
766,2019,0.036189,0.557227,0.208197,0.235884,1572.0,650000.0,1908.0,315.0


In [71]:
df_bk = df[df.GEOID=='36047033100']  # BK (crown heights)
df_bk[['year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
656,2010,0.174421,0.092822,0.084405,0.008183,0.678747,0.311667,0.0,0.028525,0.073884,0.046996,0.122516,34005.0,912.0,27.9,735800.0,0.0,0.0,222.0,1.0,13.25
656,2011,0.191112,0.058824,0.089905,0.008990,0.625225,0.349088,0.0,0.042384,0.083997,0.040843,0.131004,29571.0,912.0,27.7,768900.0,0.0,0.0,220.0,1.0,13.50
656,2012,0.175975,0.059461,0.115070,0.007222,0.684160,0.287193,0.0,0.038517,0.098941,0.046702,0.116996,32970.0,1038.0,25.8,743800.0,0.0,0.0,218.0,1.0,13.75
656,2013,0.190109,0.079614,0.106393,0.029916,0.652352,0.293848,0.0,0.065862,0.093366,0.057419,0.092159,37949.0,1046.0,27.9,820200.0,0.0,0.0,216.0,1.0,14.00
656,2014,0.156322,0.072414,0.142759,0.023448,0.629655,0.325977,0.0,0.067816,0.091954,0.053333,0.088276,37500.0,1148.0,27.3,783600.0,0.0,0.0,214.0,1.0,14.25
656,2015,0.158291,0.074235,0.148470,0.036775,0.601645,0.348333,0.0,0.068753,0.090452,0.060987,0.105528,39301.0,1225.0,28.4,894100.0,0.0,0.0,213.0,1.0,14.50
656,2016,0.165106,0.103746,0.161163,0.041646,0.603499,0.369640,0.0,0.040414,0.091917,0.089207,0.107935,37405.0,1221.0,29.6,927700.0,0.0,0.0,211.0,1.0,14.75
656,2017,0.173999,0.117400,0.136925,0.050914,0.599357,0.387049,0.0,0.043994,0.083045,0.092437,0.109244,42438.0,1257.0,34.6,948100.0,0.0,0.0,209.0,1.0,15.00
656,2018,0.163632,0.106545,0.144685,0.035433,0.611959,0.385827,0.0,0.031742,0.097933,0.084154,0.102608,39618.0,1354.0,30.5,1016200.0,0.0,0.0,207.0,1.0,9.00
656,2019,0.152538,0.103499,0.138245,0.030064,0.639724,0.357812,0.0,0.038196,0.115081,0.072942,0.073189,40216.0,1391.0,28.7,1056800.0,0.0,0.0,205.0,0.0,12.00


In [72]:
df_bk = features[features.GEOID=='36047033100']   # BK (crown heights)
df_bk[['pred_year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
656,2020,-0.004212,0.052622,0.122828,0.588142,-0.017880,0.034171,0.0,0.122647,0.040531,0.129513,-0.012305,0.020587,0.051198,0.011113,0.041505,0.0,0.0,-0.008433,0.000000,0.018035
2981,2021,-0.011183,0.135604,0.086902,0.608794,-0.005881,0.022009,0.0,0.056433,0.001630,0.157370,-0.021831,0.064744,0.056112,0.040461,0.037672,0.0,0.0,-0.008511,0.000000,0.017715
5306,2022,-0.007913,0.118388,0.049696,0.590901,-0.018088,0.051033,0.0,0.025223,0.001858,0.118524,-0.014132,0.034512,0.045947,0.032143,0.055084,0.0,0.0,-0.008591,0.000000,-0.052038
7631,2023,-0.032598,0.057135,0.054845,0.041929,-0.002777,0.035070,0.0,-0.059215,0.040434,0.058075,-0.026538,0.011858,0.049217,0.008741,0.044624,0.0,0.0,-0.008672,-0.166667,0.000487
9956,2024,0.030373,0.050423,-0.019826,-0.027524,-0.020129,0.056032,0.0,-0.069604,0.064615,0.054571,-0.017357,0.058381,0.034642,0.012906,0.027995,0.0,0.0,-0.007942,-0.200000,-0.141378
12281,2025,0.026638,-0.015320,-0.042813,-0.197101,-0.022143,0.045736,0.0,-0.055743,0.063440,-0.022664,-0.022714,0.041963,0.046357,0.002140,0.030095,0.0,0.0,-0.008797,-0.400000,-0.310968
14606,2026,0.029143,-0.102505,-0.058577,0.438132,-0.043587,0.052589,0.0,0.026778,0.070641,-0.086353,0.000902,0.049704,0.049822,-0.001344,0.027251,0.0,0.0,-0.008882,-0.500000,-0.376610
16931,2027,0.004130,-0.080114,-0.026884,0.412515,-0.044066,0.040935,0.0,0.079309,0.065590,-0.070201,-0.000678,0.034134,0.042345,-0.026593,0.022036,0.0,0.0,-0.008969,-0.666667,-0.130000


## Brooklyn - Bed-Stuy (Gentrified)

In [25]:
df_bk = df[df.GEOID=='36047022900']  # BK (Bed-Stuy)
df_bk[['year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
559,2010,0.392022,0.047990,0.021190,0.012153,0.208165,0.636335,0.045809,0.084138,0.177314,0.113119,0.075725,51711.0,1261.0,31.5,636900.0,11.0,0.0,226.0,0.0,0.75
559,2011,0.400854,0.053386,0.021049,0.008847,0.235204,0.600061,0.057962,0.106162,0.185479,0.100366,0.081147,52426.0,1305.0,31.6,662200.0,11.0,0.0,236.0,0.0,1.50
559,2012,0.357577,0.068031,0.022677,0.013551,0.238108,0.600111,0.061394,0.106195,0.193584,0.113662,0.080199,57650.0,1345.0,30.1,661000.0,11.0,0.0,246.0,0.0,2.25
559,2013,0.373043,0.053132,0.024049,0.032159,0.275727,0.589206,0.041387,0.091443,0.196868,0.141219,0.055089,62000.0,1367.0,31.8,666500.0,11.0,3754.0,256.0,0.0,3.00
559,2014,0.393000,0.063585,0.017366,0.033396,0.278921,0.583222,0.045952,0.078012,0.226556,0.118889,0.063585,61250.0,1387.0,31.8,677500.0,11.0,4311.0,266.0,0.0,3.75
559,2015,0.372751,0.064286,0.024339,0.042593,0.329101,0.524339,0.078042,0.052910,0.216931,0.103968,0.080952,51645.0,1505.0,31.0,633300.0,11.0,5105.0,277.0,0.0,4.50
559,2016,0.378596,0.079860,0.031729,0.037645,0.319978,0.515730,0.099220,0.049745,0.264587,0.123151,0.102716,60625.0,1629.0,32.7,619700.0,11.0,7839.0,287.0,0.0,5.25
559,2017,0.419472,0.063439,0.030813,0.052046,0.387105,0.474625,0.085707,0.073796,0.291041,0.151476,0.107198,64935.0,1792.0,32.6,674200.0,11.0,10994.0,297.0,0.0,6.00
559,2018,0.409529,0.071856,0.033064,0.047383,0.397292,0.468888,0.086696,0.077324,0.300182,0.164540,0.109867,76591.0,2073.0,32.4,994000.0,11.0,15328.0,307.0,0.0,3.00
559,2019,0.397279,0.071293,0.029932,0.049796,0.445442,0.429660,0.082449,0.077007,0.321905,0.180136,0.108571,83043.0,2114.0,33.8,1159100.0,11.0,20548.0,317.0,0.0,11.00


In [74]:
df_bk = features[features.GEOID=='36047022900']  # BK (Bed-Stuy)
df_bk[['pred_year','percent_public_transit',
       'percent_drive_commute', 'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_drive_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
596,2020,0.006659,-0.022043,-0.057763,-0.017038,0.200663,-0.044510,0.226147,0.171196,0.067701,0.089169,-0.059735,-0.047207,0.011104,-0.004504,0.031513,0.0,0.000000,0.043187,0.000000,0.176550
2921,2021,-0.028623,-0.045408,-0.040768,0.170713,0.130124,-0.032817,0.023754,0.193553,0.035208,0.077788,-0.018952,-0.017174,0.012165,0.012148,0.058756,0.0,0.000000,0.041313,0.000000,0.148337
5246,2022,-0.032699,-0.039610,0.019904,0.225115,0.131092,-0.041556,0.049287,0.204902,0.070202,0.049639,0.034242,0.019951,0.039545,-0.002372,0.100908,0.0,0.000000,0.039738,-0.166667,0.140401
7571,2023,-0.031887,-0.053683,0.161143,0.278535,0.102385,-0.032925,-0.023187,0.155273,0.120039,-0.006522,0.068286,0.060940,0.063003,0.001006,0.123737,0.0,0.000000,0.038146,-0.200000,0.015751
9896,2024,-0.046260,-0.082610,0.176491,0.436662,0.125256,-0.052766,0.150941,0.067698,0.180448,-0.015950,0.051418,0.111475,0.084309,-0.004606,0.151935,0.0,0.000000,0.036800,-0.250000,-0.148138
12221,2025,-0.036555,-0.000672,0.297318,0.510504,0.112016,-0.055475,0.149699,0.080004,0.170698,0.005851,0.024705,0.154004,0.101993,-0.007312,0.158736,0.0,0.000000,0.035433,-0.333333,-0.336544
14546,2026,-0.022274,0.020463,0.384912,0.453690,0.093950,-0.071236,0.144374,0.030503,0.097688,0.020292,0.086906,0.190921,0.107779,-0.013099,0.115101,0.0,0.000000,0.034269,-0.500000,-0.426929
16871,2027,0.000922,0.005070,0.330547,0.258077,0.054708,-0.052923,0.144661,0.015725,0.095689,0.014392,0.056211,0.139457,0.096310,-0.009111,0.086490,0.0,3.342466,0.033080,-1.000000,-0.559524


In [93]:
df_bronx = df[df.GEOID=='36005006301']  # Bronx - Yankee Stadium
df_bronx[['total_pop','year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,total_pop,year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
40,5165.117474,2010,0.255423,0.041840,0.012562,0.124017,0.532288,0.008794,0.502769,0.077194,0.036975,0.195407,28846.207921,898.040181,33.021038,367071.619401,73.0,0.0,509.0,2.0,22.125
40,5253.513688,2011,0.259523,0.033394,0.014702,0.135887,0.532525,0.010321,0.501121,0.079979,0.045539,0.197961,28723.295205,868.916300,32.841917,390813.625242,73.0,0.0,543.0,2.0,26.250
40,5288.371080,2012,0.260444,0.025291,0.012261,0.141450,0.514421,0.009975,0.504426,0.114914,0.069668,0.197695,29509.181982,900.435234,32.921088,354218.766010,73.0,0.0,576.0,2.0,30.375
40,5250.015671,2013,0.259385,0.036386,0.015734,0.148560,0.500568,0.014373,0.508460,0.112217,0.067070,0.194902,29314.137694,928.863378,33.315989,351426.654082,73.0,0.0,610.0,2.0,34.500
40,5435.005838,2014,0.264942,0.034461,0.018198,0.135977,0.491520,0.011846,0.508655,0.075794,0.059774,0.191916,28883.711560,1040.279013,33.083828,341818.043473,73.0,0.0,644.0,2.0,38.625
40,5521.871893,2015,0.273686,0.040807,0.018468,0.123391,0.482878,0.010043,0.509985,0.088334,0.059003,0.190091,29454.122012,1352.478365,32.933545,315243.413959,73.0,0.0,678.0,2.0,42.750
40,4829.833374,2016,0.284640,0.042461,0.019197,0.119632,0.464438,0.011015,0.514620,0.095156,0.053888,0.189117,29735.908982,1068.975045,32.902242,295230.050913,73.0,0.0,711.0,2.0,46.875
40,5558.440332,2017,0.294723,0.041815,0.017904,0.127767,0.466102,0.014459,0.507508,0.112889,0.060322,0.185422,31228.897977,1085.194215,33.239889,291733.016586,73.0,0.0,745.0,2.0,51.000
40,5492.170435,2018,0.303367,0.032104,0.017751,0.134373,0.448446,0.010040,0.514232,0.115417,0.077967,0.182688,33973.216101,1501.717161,33.676130,344142.527358,79.0,0.0,779.0,1.0,40.000
40,5506.101544,2019,0.314219,0.028612,0.015525,0.144883,0.440992,0.026685,0.519025,0.125613,0.080457,0.178039,36726.392378,1190.424101,34.044061,415663.025120,79.0,0.0,812.0,3.0,36.000


In [94]:
df_change_bronx = features[features.GEOID=='36005006301']  # Bronx - Yankee Stadium
df_change_bronx[['pred_year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
40,2020,0.018331,0.027660,0.083081,-0.003469,-0.022388,0.058296,0.003900,0.061252,0.085966,-0.005392,0.005187,0.040976,-0.000580,-0.034235,0.000000,0.000000,0.057301,0.000000,0.133706
2365,2021,0.021559,0.058768,0.043464,-0.008087,-0.021865,0.081469,0.002143,0.086299,0.067262,-0.010827,0.014265,0.048910,0.002034,-0.046990,0.000000,0.000000,0.054138,0.000000,0.117300
4690,2022,0.025856,0.060505,0.069718,-0.006292,-0.022512,0.036119,0.003252,0.017230,0.027709,-0.013061,0.024351,0.106835,0.003820,-0.001442,0.013699,0.000000,0.051615,-0.083333,0.055161
7015,2023,0.032496,-0.030740,0.001604,-0.001634,-0.020794,0.238967,0.003473,0.035865,0.039248,-0.014947,0.038960,0.067024,0.003642,0.034509,0.013699,0.000000,0.048837,0.250000,0.015861
9340,2024,0.045740,0.161572,-0.037185,0.206621,-0.081534,0.254417,0.029642,0.052031,-0.010107,-0.012805,0.093071,0.067695,0.012910,0.007044,0.013699,0.000000,0.046527,0.083333,-0.156844
11665,2025,0.028386,0.173120,0.050451,0.187721,-0.027320,0.262416,0.016262,0.029535,-0.071223,-0.010290,0.097269,0.021911,0.017869,0.014442,0.034796,2.480047,0.044426,0.100000,-0.341310
13990,2026,0.025699,0.176805,0.197220,0.174859,-0.004409,0.255263,0.001804,0.061327,-0.016096,0.015014,0.112659,0.084423,0.026224,0.033649,0.034796,1.295836,0.042563,0.125000,-0.428871
16315,2027,0.005332,0.177691,0.254415,0.183142,0.035757,0.152040,-0.021952,0.028459,0.019083,0.005143,0.112480,0.086376,0.023645,0.041335,0.034796,0.988883,0.040800,0.166667,0.453529


In [96]:
df_bronx = df[df.GEOID=='36005001600']  # Bronx - Clason Point
df_bronx[['total_pop','year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,total_pop,year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
3,5140.0,2010,0.172568,0.007004,0.006226,0.191245,0.357977,0.000000,0.621401,0.046109,0.042218,0.094553,30355.0,823.0,40.2,378100.0,12.0,0.0,367.0,0.0,8.125
3,5263.0,2011,0.206346,0.015200,0.000380,0.161125,0.403002,0.000000,0.600228,0.080182,0.037241,0.099753,33910.0,837.0,39.1,455200.0,12.0,0.0,394.0,0.0,9.250
3,5127.0,2012,0.180222,0.013458,0.000390,0.179832,0.370782,0.000000,0.652038,0.056758,0.027697,0.114687,32328.0,856.0,39.9,413800.0,12.0,0.0,421.0,0.0,10.375
3,5215.0,2013,0.153979,0.015340,0.000000,0.294343,0.329243,0.000000,0.707958,0.054267,0.023202,0.158198,30817.0,863.0,39.2,410300.0,12.0,0.0,448.0,0.0,11.500
3,5412.0,2014,0.169623,0.027901,0.004250,0.342757,0.333703,0.000000,0.682557,0.065965,0.028640,0.137472,32354.0,920.0,39.9,418300.0,17.0,0.0,475.0,0.0,12.625
3,5879.0,2015,0.162613,0.030617,0.006123,0.389692,0.374043,0.000000,0.651301,0.057153,0.026025,0.199184,32312.0,950.0,38.8,390100.0,17.0,0.0,503.0,0.0,13.750
3,6056.0,2016,0.150264,0.023448,0.012219,0.433124,0.360799,0.000000,0.667272,0.034841,0.024273,0.189564,31540.0,945.0,36.9,376400.0,17.0,0.0,530.0,0.0,14.875
3,6115.0,2017,0.162061,0.025184,0.013246,0.439084,0.379395,0.001799,0.633361,0.043500,0.030744,0.179395,33118.0,989.0,32.6,389600.0,17.0,0.0,557.0,0.0,16.000
3,5917.0,2018,0.187257,0.019605,0.025689,0.406625,0.411357,0.002197,0.608923,0.077404,0.032787,0.123204,35802.0,1065.0,37.4,406800.0,17.0,0.0,584.0,0.0,10.000
3,5825.0,2019,0.183863,0.010815,0.019399,0.323433,0.435880,0.013047,0.592618,0.079485,0.032103,0.128240,34093.0,1030.0,33.7,490000.0,17.0,0.0,611.0,0.0,11.000


In [97]:
df_change_bronx = features[features.GEOID=='36005001600']  # Bronx - Clason Point
df_change_bronx[['pred_year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
3,2020,-0.015358,0.312918,-0.095215,0.168040,0.005470,0.000000,0.013476,0.015757,-0.076782,0.142282,0.008068,0.023547,-0.013855,0.003578,0.069444,0.000000,0.063187,0.0,0.106212
2328,2021,-0.034897,0.130211,0.109384,0.196583,-0.006903,0.000000,0.010685,-0.065989,-0.012706,0.124174,-0.003113,0.028472,-0.028716,-0.024563,0.069444,0.000000,0.059416,0.0,0.095740
4653,2022,0.012116,0.112390,0.291947,0.164911,0.020463,0.221365,-0.010132,0.112603,0.041084,0.047019,0.018170,0.037496,-0.007586,-0.002046,0.069444,0.000000,0.056074,0.0,0.012970
6978,2023,0.033364,0.014361,0.442979,0.024685,0.049071,2.579927,-0.028889,0.124401,0.064652,-0.009399,0.018004,0.030656,-0.021151,0.033451,0.069444,0.000000,0.053091,0.0,0.011564
9303,2024,0.023951,-0.192410,0.372366,-0.053218,0.032202,1.714768,-0.031061,0.101475,-0.020870,-0.061980,0.012982,0.007350,0.003569,0.059180,0.000000,0.000000,0.050684,0.0,-0.171407
11628,2025,0.029941,0.110262,0.334632,-0.120763,0.006563,1.245607,-0.025349,0.127164,0.072076,-0.140526,0.038448,0.021307,0.017917,0.074009,0.000000,0.000000,0.047901,0.0,-0.223510
13953,2026,0.025294,0.172199,0.202046,-0.215878,0.006608,1.241623,-0.026261,0.235994,0.014375,-0.116786,0.047963,0.029383,0.035294,0.117700,0.000000,0.000000,0.045712,0.0,-0.299842
16278,2027,-0.030618,0.200588,0.205152,-0.304021,0.007478,1.194357,-0.023346,0.151465,0.099868,-0.064898,0.046051,0.019523,0.047882,0.105637,0.088235,1.916667,0.043715,0.0,0.681250


In [101]:
df_man = df[df.GEOID=='36061023501']  # Mannhattan - Washington Heights
df_man[['total_pop','year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,total_pop,year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
1428,7018.0,2010,0.249644,0.046594,0.018809,0.124394,0.621972,0.013537,0.328441,0.077087,0.057281,0.185523,34679.0,938.0,34.8,242300.0,24.0,0.0,186.0,1.875,12.875
1428,6709.0,2011,0.270532,0.042927,0.019526,0.157251,0.582352,0.016694,0.331942,0.072887,0.059920,0.178566,38158.0,933.0,34.6,349200.0,24.0,0.0,200.0,1.750,14.750
1428,6855.0,2012,0.257330,0.029468,0.021152,0.113056,0.620277,0.018964,0.301823,0.085047,0.057039,0.148213,38678.0,918.0,32.9,353000.0,36.0,0.0,214.0,1.625,16.625
1428,6499.0,2013,0.312663,0.015695,0.018618,0.142022,0.515926,0.023542,0.341129,0.115402,0.074781,0.171257,39913.0,946.0,35.2,268800.0,36.0,0.0,228.0,1.500,18.500
1428,6383.0,2014,0.321636,0.019740,0.010967,0.168103,0.491462,0.027260,0.372082,0.101833,0.087890,0.175153,33813.0,930.0,36.1,251500.0,36.0,0.0,242.0,1.375,20.375
1428,6647.0,2015,0.295622,0.017451,0.020159,0.140515,0.475854,0.034753,0.431774,0.088762,0.081691,0.203250,34846.0,960.0,31.7,275400.0,37.0,0.0,256.0,1.250,22.250
1428,6132.0,2016,0.315232,0.018754,0.038324,0.212003,0.433790,0.025277,0.417319,0.117580,0.099152,0.184279,36190.0,990.0,32.6,244400.0,37.0,0.0,269.0,1.125,24.125
1428,6820.0,2017,0.338270,0.033578,0.045015,0.266862,0.408504,0.016276,0.404399,0.129179,0.094575,0.210411,40145.0,1095.0,31.5,277700.0,37.0,0.0,283.0,1.000,26.000
1428,6776.0,2018,0.405844,0.035419,0.047226,0.281287,0.377066,0.023465,0.433442,0.154664,0.124705,0.238489,45912.0,1260.0,31.1,317800.0,37.0,0.0,297.0,3.000,16.000
1428,7044.0,2019,0.427740,0.036059,0.049120,0.276973,0.422487,0.023424,0.404458,0.159426,0.127484,0.185974,59405.0,1369.0,31.1,403100.0,41.0,0.0,311.0,0.000,15.000


In [103]:
df_change_man = features[features.GEOID=='36061023501']  # Mannhattan - Washington Heights
df_change_man[['pred_year','percent_public_transit',
       'percent_walk_commute',
       'percent_wfh', 'percent_white', 'percent_black', 
       'percent_asian', 'percent_latino',
       'percent_bachelors', 'percent_grad','percent_not_citizen',
       'med_income','med_rent_acs', 'med_age', 'med_value',
        'num_bikelanes','num_citibike_rides', 'num_trees',
       'num_evictions_commercial', 'num_evictions_residential']]

,pred_year,percent_public_transit,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_asian,percent_latino,percent_bachelors,percent_grad,percent_not_citizen,med_income,med_rent_acs,med_age,med_value,num_bikelanes,num_citibike_rides,num_trees,num_evictions_commercial,num_evictions_residential
1428,2020,0.044008,-0.107198,0.221654,0.127930,-0.055730,0.128460,0.044640,0.091332,0.104591,0.006304,0.010361,0.009281,-0.008816,0.021942,0.087963,0.000000,0.063454,-0.081543,0.110530
3753,2021,0.042244,0.037656,0.244398,0.127035,-0.054829,0.030234,0.037703,0.116855,0.089221,0.036188,0.011855,0.027847,-0.013482,-0.028881,0.087963,0.000000,0.059584,-0.088951,0.099211
6078,2022,0.083671,0.099055,0.238701,0.182885,-0.078509,0.081190,0.064795,0.121929,0.150332,0.086759,0.033526,0.055640,-0.007409,-0.006628,0.004630,0.000000,0.056162,0.256287,0.013922
8403,2023,0.056825,0.179964,0.265354,0.137628,-0.030394,0.040667,0.031945,0.067575,0.102204,0.024146,0.077186,0.064975,-0.019061,0.077861,0.022648,0.000000,0.053115,0.102441,-0.015292
10728,2024,0.055062,0.189427,0.473267,0.132439,-0.058499,0.061630,0.016365,0.098742,0.103266,0.030641,0.130349,0.115517,-0.014748,0.103059,0.022648,0.000000,0.050919,0.139596,-0.154406
13053,2025,0.048709,0.170240,0.433469,0.115534,-0.036324,0.264590,-0.018165,0.137379,0.163926,-0.003960,0.130832,0.117712,0.006076,0.079574,0.018018,1.185570,0.047924,0.197222,-0.336410
15378,2026,0.025392,0.108538,0.277434,-0.025271,-0.026398,0.302452,0.012618,0.064922,0.128522,0.004149,0.137510,0.113771,0.007950,0.152760,0.018018,0.723154,0.046835,0.296296,-0.420546
17703,2027,-0.007630,0.043485,0.267669,-0.046490,-0.019461,0.358082,0.010813,0.040629,0.145110,-0.041309,0.126019,0.093129,0.021394,0.136785,0.018018,0.607065,0.044752,0.500000,0.563910


## Mannhattan - East Harlem (Declining)

In [39]:
df_man = df[df.GEOID=='36061015602']  # Mannhattan - East Harlem
df_man[['year', 'percent_hh_english',
       'percent_work_edu_health', 'percent_asian', 
       'med_rent_acs', 'med_hh_age',
        'med_hh_tenure', 'num_citibike_rides']]

,year,percent_hh_english,percent_work_edu_health,percent_asian,med_rent_acs,med_hh_age,med_hh_tenure,num_citibike_rides
1346,2010,0.342078,0.222591,0.179314,675.0,68.0,20.0,0.0
1346,2011,0.335650,0.213701,0.281128,981.0,66.0,20.0,0.0
1346,2012,0.306584,0.240586,0.214523,996.0,68.0,22.0,0.0
1346,2013,0.278557,0.199153,0.229551,961.0,68.0,22.0,0.0
1346,2014,0.311839,0.127000,0.287402,950.0,69.0,21.0,0.0
1346,2015,0.349474,0.130475,0.212900,979.0,67.0,20.0,0.0
1346,2016,0.424565,0.223794,0.138142,963.0,58.0,18.0,4059.0
1346,2017,0.494717,0.192338,0.138708,987.0,57.0,17.0,8132.0
1346,2018,0.505213,0.254335,0.117982,1013.0,63.0,16.0,9583.0
1346,2019,0.496262,0.324913,0.115084,861.0,58.0,14.0,13537.0


In [38]:
df_man.drop('NAME',axis=1)

,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,year,med_hh_age,med_hh_tenure,full_tract,GEOID,num_citibike_rides,num_trees,med_rent,num_affordable_hous_built,num_bikelanes,num_evictions_commercial,num_evictions_residential,park_acres,num_parks,random_lat,random_lon
1346,2069.0,0.034627,0.144806,0.342078,0.0,0.053156,0.052049,0.199336,0.112957,0.222591,0.170543,0.043016,0.122281,0.354277,0.025133,0.000000,0.043016,0.009183,0.378927,0.178830,0.0,0.179314,0.000000,0.428710,0.072015,0.219913,0.679555,0.145964,0.088932,0.693088,0.069116,39375.0,675.0,30.7,0.253935,1000001.0,0.463799,0.0,2010,68.0,20.0,061015602,36061015602,0.0,57.0,1991.818875,0.0,0.0,0.0,9.25,1.478,1.0,225931.33149,998694.443668
1346,2305.0,0.117180,0.214499,0.335650,0.0,0.026585,0.175869,0.186094,0.131902,0.213701,0.087935,0.072451,0.098915,0.354447,0.019089,0.000000,0.030803,0.014317,0.411714,0.172234,0.0,0.281128,0.000000,0.360521,0.124946,0.209111,0.609544,0.240781,0.074187,0.711063,0.068547,44896.0,981.0,32.5,0.249255,1000001.0,0.370407,0.0,2011,66.0,20.0,061015602,36061015602,0.0,58.0,2039.311287,0.0,0.0,0.0,8.50,1.478,1.0,225931.33149,998694.443668
1346,2107.0,0.074074,0.277778,0.306584,0.0,0.032427,0.104603,0.190377,0.111925,0.240586,0.125523,0.026578,0.109635,0.355482,0.023256,0.000000,0.049834,0.019934,0.403892,0.175605,0.0,0.214523,0.000000,0.439487,0.118178,0.189843,0.623636,0.176554,0.125297,0.788325,0.046986,45000.0,996.0,35.2,0.000000,1000001.0,0.389918,0.0,2012,68.0,22.0,061015602,36061015602,0.0,59.0,2087.936096,0.0,0.0,0.0,7.75,1.478,1.0,225931.33149,998694.443668
1346,2274.0,0.088176,0.322645,0.278557,0.0,0.046610,0.137712,0.145127,0.156780,0.199153,0.130297,0.023307,0.065084,0.319261,0.025506,0.000000,0.043096,0.021548,0.400616,0.205365,0.0,0.229551,0.000000,0.439314,0.124890,0.149956,0.613017,0.213720,0.100704,0.861478,0.032102,44643.0,961.0,38.3,0.000000,1000001.0,0.386774,0.0,2013,68.0,22.0,061015602,36061015602,0.0,60.0,2137.720302,0.0,0.0,0.0,7.00,1.478,1.0,225931.33149,998694.443668
1346,2286.0,0.081395,0.254757,0.311839,0.0,0.046000,0.172000,0.121000,0.201000,0.127000,0.109000,0.019248,0.047682,0.341645,0.026684,0.000000,0.052493,0.016623,0.334646,0.188539,0.0,0.287402,0.000000,0.393263,0.112861,0.117673,0.610236,0.209099,0.125109,0.912073,0.011374,45592.0,950.0,38.3,0.000000,1000001.0,0.385835,0.0,2014,69.0,21.0,061015602,36061015602,0.0,61.0,2188.691550,0.0,0.0,0.0,6.25,1.478,1.0,225931.33149,998694.443668
1346,2217.0,0.053684,0.201053,0.349474,0.0,0.010252,0.152842,0.115564,0.270270,0.130475,0.122088,0.009923,0.041949,0.395129,0.027064,0.000000,0.042400,0.016238,0.401443,0.221470,0.0,0.212900,0.000000,0.406856,0.123590,0.125846,0.651331,0.164186,0.119982,0.907533,0.009472,50455.0,979.0,37.0,0.917895,1046900.0,0.372632,0.0,2015,67.0,20.0,061015602,36061015602,0.0,62.0,2272.020913,0.0,0.0,0.0,5.50,1.478,1.0,225931.33149,998694.443668
1346,2454.0,0.024178,0.214700,0.424565,0.0,0.009983,0.076539,0.112313,0.176373,0.223794,0.163062,0.004075,0.041157,0.341891,0.045232,0.000000,0.083537,0.014262,0.497963,0.224124,0.0,0.138142,0.016707,0.416463,0.097800,0.192339,0.759169,0.091280,0.105542,0.886308,0.015077,54458.0,963.0,37.0,0.899420,995300.0,0.492263,0.0,2016,58.0,18.0,061015602,36061015602,4059.0,62.0,2303.308283,0.0,8.0,0.0,4.75,1.478,1.0,225931.33149,998694.443668
1346,2617.0,0.027858,0.215178,0.49